In [12]:
%pwd

'c:\\Arjun_Works\\NextStep-AI'

In [2]:
import os

os.chdir("../")

%pwd

'c:\\Arjun_Works\\NextStep-AI'

In [13]:
from langchain_community.document_loaders import PyPDFLoader

In [14]:
import glob

pdf_files = glob.glob("data/*.pdf")
loaders = [PyPDFLoader(pdf) for pdf in pdf_files]

In [18]:
documents = []
for loader in loaders:
    documents.extend(loader.load())

In [19]:
documents[0].metadata

{'producer': '3.0.24 (5.1.10)',
 'creator': 'Microsoft Word',
 'creationdate': '2025-03-31T21:43:39-07:00',
 'author': 'Dhaval Patel',
 'moddate': '2025-04-01T06:46:53+02:00',
 'source': 'data\\ai_engineer_roadmap_2025_v2.pdf',
 'total_pages': 15,
 'page': 0,
 'page_label': '1'}

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [ ]:
chunks = text_splitter.split_documents(documents)

In [ ]:
chunks

[Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'Microsoft Word', 'creationdate': '2025-03-31T21:43:39-07:00', 'author': 'Dhaval Patel', 'moddate': '2025-04-01T06:46:53+02:00', 'source': 'data\\ai_engineer_roadmap_2025_v2.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='codebasics.io \n \n1 \nAI Engineer Roadmap for Beginners \nFollowing is the roadmap to learning AI Engineer (also known as ML Engineer) skills for a total \nbeginner. It includes FREE learning resources for technical skills (or tool skills) and soft (or core) skills \n                       \nFind Your Suitability: Before you start your learning journey, it is important you find out if AI \nengineering career really suits your natural abilities and interests. Take this test to know your \nsuitability: https://codebasics.io/survey/find-your-match-ds \nProceed further if the results show that this career role matches you. \nTotal Duration: 8 Months (4 hours of study every day, 6 days a w

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'Page \d+ of \d+', '', text)  
    return text.strip()


In [ ]:
from langchain.docstore.document import Document

cleaned_chunks = [
    Document(page_content=clean_text(doc.page_content), metadata=doc.metadata)
    for doc in chunks
]


In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
)

In [ ]:
from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("knowledge_index")

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.load_local("knowledge_index", embedding, allow_dangerous_deserialization=True)

In [ ]:
resume_text = """
Experienced B.Tech IT student with hands-on skills in web development, React, Python, Firebase, and GenAI tools. Completed 20+ full-stack projects and internship building ReArticle (LaTeX paper generator). Strong in LangChain, HuggingFace, and AI/ML.
"""

traits = """
Fast learner, creative, adaptable, interest in AI/ML and Full Stack Development
"""

duration = "6-month"


In [ ]:
domain_prompt = PromptTemplate(
    input_variables=["resume", "traits"],
    template="""
You are a career coach AI.

Given this resume:
{resume}

And these personal traits:
{traits}

Suggest 3 most suitable tech domains this person should explore. Be concise and justify each suggestion in one line.
"""
)

llm = init_chat_model("google_genai:gemini-2.0-flash")

domain_chain = LLMChain(prompt=domain_prompt, llm=llm)
domain_result = domain_chain.run({"resume": resume_text, "traits": traits})
print("🧭 Recommended Domains:\n", domain_result)


🧭 Recommended Domains:
 Okay, based on the resume and personal traits, here are 3 suitable tech domains:

1.  **AI-Powered Web Development:** Leverages existing full-stack skills with a passion for AI/ML to build intelligent and personalized web applications.
2.  **Generative AI Application Development:** Directly uses expertise in GenAI tools, LangChain, and HuggingFace to create innovative applications using generative models.
3.  **Full-Stack AI/ML Engineering:** Integrates full-stack development skills with AI/ML knowledge to build and deploy complete AI-driven systems from front-end to back-end.


In [ ]:
import re

def extract_first_domain(text):
    lines = text.strip().split("\n")
    for line in lines:
        match = re.match(r"(\d+\. )?([A-Za-z /+-]+)", line)
        if match:
            return match.group(2).strip()
    return "Full Stack Development"  

recommended_domain = extract_first_domain(domain_result)
print("✅ Picked Domain:", recommended_domain)


✅ Picked Domain: Okay


In [ ]:
roadmap_prompt = PromptTemplate(
    input_variables=["domain", "duration"],
    template="""
Create a {duration} learning roadmap for a complete beginner in {domain}.

Break it into phases or months. For each, suggest goals, tools, projects, and top resources.
Output in markdown-friendly format.
"""
)

roadmap_chain = LLMChain(prompt=roadmap_prompt, llm=llm)
roadmap_result = roadmap_chain.run({"domain": recommended_domain, "duration": duration})
print("📈 Roadmap:\n", roadmap_result)


📈 Roadmap:
 Okay, here's a 6-month learning roadmap for a complete beginner in Okay, broken down into monthly phases with goals, tools, projects, and resources. This roadmap assumes a commitment of roughly 5-10 hours per week.  Adjust as needed based on your available time and learning pace.  "Okay" is a bit ambiguous, so I'm assuming you mean the programming language **OCaml**. If you meant something else, please clarify and I'll adjust.

```markdown
# 6-Month OCaml Learning Roadmap for Beginners

This roadmap guides you from zero knowledge to basic proficiency in OCaml programming.

## Assumptions:

*   **Time Commitment:** 5-10 hours per week.
*   **Prerequisites:** Basic understanding of general programming concepts (variables, loops, functions) is helpful but not required.  We'll cover the basics.
*   **Goal:** To be able to develop small, functional OCaml programs and understand more complex code.

## Month 1: Foundations - Getting Started

**Goal:** Set up your environment, unde

In [ ]:
retrieved_docs = db.similarity_search(recommended_domain, k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Snippet {i+1} ---")
    print(doc.page_content[:500]) 


--- Snippet 1 ---
Do you believe there is a difference?
 3. Suppose a computerized medical expert system gains a reputation within the 
medical community for giving sound advice. To what extent should a physi-
cian allow that system to alter his or her decisions regarding the treatment 
of patients? If the physician applies a treatment contrary to that proposed by 
the expert system and the system turns out to be right, is the physician guilty 
of malpractice? In general, if an expert system becomes well-known wi

--- Snippet 2 ---
263Chapter Review Problems
 6. The following program segment is designed to compute the product of 
two nonnegative integers X and Y by accumulating the sum of X copies of 
Y— that is, 3 times 4 is computed by accumulating the sum of three 4s. Is 
the program correct? Justify your answer.
Product = Y
Count = 1
while (Count < X):
  Product = Product + Y
  Count = Count + 1
 7. Assuming the precondition that the value associated with N is a posi -
tive intege

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [21]:
chunks = text_splitter.split_documents(documents)

In [22]:
chunks

[Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'Microsoft Word', 'creationdate': '2025-03-31T21:43:39-07:00', 'author': 'Dhaval Patel', 'moddate': '2025-04-01T06:46:53+02:00', 'source': 'data\\ai_engineer_roadmap_2025_v2.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='codebasics.io \n \n1 \nAI Engineer Roadmap for Beginners \nFollowing is the roadmap to learning AI Engineer (also known as ML Engineer) skills for a total \nbeginner. It includes FREE learning resources for technical skills (or tool skills) and soft (or core) skills \n                       \nFind Your Suitability: Before you start your learning journey, it is important you find out if AI \nengineering career really suits your natural abilities and interests. Take this test to know your \nsuitability: https://codebasics.io/survey/find-your-match-ds \nProceed further if the results show that this career role matches you. \nTotal Duration: 8 Months (4 hours of study every day, 6 days a w

In [23]:
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
)

In [25]:
from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("knowledge_index")

In [37]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.load_local("knowledge_index", embedding, allow_dangerous_deserialization=True)

In [35]:
resume_text = """
Experienced B.Tech IT student with hands-on skills in web development, React, Python, Firebase, and GenAI tools. Completed 20+ full-stack projects and internship building ReArticle (LaTeX paper generator). Strong in LangChain, HuggingFace, and AI/ML.
"""

traits = """
Fast learner, creative, adaptable, interest in AI/ML and Full Stack Development
"""

duration = "6-month"


In [39]:
domain_prompt = PromptTemplate(
    input_variables=["resume", "traits"],
    template="""
You are a career coach AI.

Given this resume:
{resume}

And these personal traits:
{traits}

Suggest 3 most suitable tech domains this person should explore. Be concise and justify each suggestion in one line.
"""
)

llm = init_chat_model("google_genai:gemini-2.0-flash")

domain_chain = LLMChain(prompt=domain_prompt, llm=llm)
domain_result = domain_chain.run({"resume": resume_text, "traits": traits})
print("🧭 Recommended Domains:\n", domain_result)


🧭 Recommended Domains:
 Okay, based on the resume and personal traits, here are 3 suitable tech domains:

1.  **AI-Powered Web Development:** Leverages existing full-stack skills with a passion for AI/ML to build intelligent and personalized web applications.
2.  **Generative AI Application Development:** Directly uses expertise in GenAI tools, LangChain, and HuggingFace to create innovative applications using generative models.
3.  **Full-Stack AI/ML Engineering:** Integrates full-stack development skills with AI/ML knowledge to build and deploy complete AI-driven systems from front-end to back-end.


In [40]:
import re

def extract_first_domain(text):
    lines = text.strip().split("\n")
    for line in lines:
        match = re.match(r"(\d+\. )?([A-Za-z /+-]+)", line)
        if match:
            return match.group(2).strip()
    return "Full Stack Development"  

recommended_domain = extract_first_domain(domain_result)
print("✅ Picked Domain:", recommended_domain)


✅ Picked Domain: Okay


In [41]:
roadmap_prompt = PromptTemplate(
    input_variables=["domain", "duration"],
    template="""
Create a {duration} learning roadmap for a complete beginner in {domain}.

Break it into phases or months. For each, suggest goals, tools, projects, and top resources.
Output in markdown-friendly format.
"""
)

roadmap_chain = LLMChain(prompt=roadmap_prompt, llm=llm)
roadmap_result = roadmap_chain.run({"domain": recommended_domain, "duration": duration})
print("📈 Roadmap:\n", roadmap_result)


📈 Roadmap:
 Okay, here's a 6-month learning roadmap for a complete beginner in Okay, broken down into monthly phases with goals, tools, projects, and resources. This roadmap assumes a commitment of roughly 5-10 hours per week.  Adjust as needed based on your available time and learning pace.  "Okay" is a bit ambiguous, so I'm assuming you mean the programming language **OCaml**. If you meant something else, please clarify and I'll adjust.

```markdown
# 6-Month OCaml Learning Roadmap for Beginners

This roadmap guides you from zero knowledge to basic proficiency in OCaml programming.

## Assumptions:

*   **Time Commitment:** 5-10 hours per week.
*   **Prerequisites:** Basic understanding of general programming concepts (variables, loops, functions) is helpful but not required.  We'll cover the basics.
*   **Goal:** To be able to develop small, functional OCaml programs and understand more complex code.

## Month 1: Foundations - Getting Started

**Goal:** Set up your environment, unde

In [42]:
retrieved_docs = db.similarity_search(recommended_domain, k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Snippet {i+1} ---")
    print(doc.page_content[:500]) 


--- Snippet 1 ---
Do you believe there is a difference?
 3. Suppose a computerized medical expert system gains a reputation within the 
medical community for giving sound advice. To what extent should a physi-
cian allow that system to alter his or her decisions regarding the treatment 
of patients? If the physician applies a treatment contrary to that proposed by 
the expert system and the system turns out to be right, is the physician guilty 
of malpractice? In general, if an expert system becomes well-known wi

--- Snippet 2 ---
263Chapter Review Problems
 6. The following program segment is designed to compute the product of 
two nonnegative integers X and Y by accumulating the sum of X copies of 
Y— that is, 3 times 4 is computed by accumulating the sum of three 4s. Is 
the program correct? Justify your answer.
Product = Y
Count = 1
while (Count < X):
  Product = Product + Y
  Count = Count + 1
 7. Assuming the precondition that the value associated with N is a posi -
tive intege